In [1]:
import os
import os.path
import numpy as np
from PIL import Image
from torchvision import transforms
import torch
from typing import List, Union, Tuple, Any
from striprtf.striprtf import rtf_to_text
import albumentations
import cv2, os, argparse, glob, PIL, tqdm
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
import os
import os.path
import numpy as np
from PIL import Image
from torchvision import transforms
import torch
from typing import List, Union, Tuple, Any
from striprtf.striprtf import rtf_to_text
import albumentations


class VideoRecord(object):
    """
    Helper class for class VideoFrameDataset. This class
    represents a video sample's metadata.

    Args:
        root_datapath: the system path to the root folder of the videos.
        row: A list with four or more elements where
             1) The first element is the path to the video sample's frames excluding
             the root_datapath prefix
             2) The second element is the starting frame id of the video
             3) The third element is the inclusive ending frame id of the video
             4) The fourth element is the label index.
             5) any following elements are labels in the case of multi-label classification
    """
    def __init__(self, row, root_datapath):
        self._data = row
        self._path = os.path.join(root_datapath, row[0])

    @property
    def path(self) -> str:
        return self._path

    @property
    def num_frames(self) -> int:
        return self.end_frame - self.start_frame + 1  # +1 because end frame is inclusive

    @property
    def start_frame(self) -> int:
        return int(self._data[1])

    @property
    def end_frame(self) -> int:
        return int(self._data[2])

    @property
    def label(self) -> Union[int, List[int]]:
        # just one label_id
        if len(self._data) == 4:
            return int(self._data[3])
        # sample associated with multiple labels
        else:
            return [int(label_id) for label_id in self._data[3:]]


class VideoFrameDataset(torch.utils.data.Dataset):
    r"""
    A highly efficient and adaptable dataset class for videos.
    Instead of loading every frame of a video,
    loads x RGB frames of a video (sparse temporal sampling) and evenly
    chooses those frames from start to end of the video, returning
    a list of x PIL images or ``FRAMES x CHANNELS x HEIGHT x WIDTH``
    tensors.

    More specifically, the frame range [START_FRAME, END_FRAME] is divided into NUM_SEGMENTS
    segments and FRAMES_PER_SEGMENT consecutive frames are taken from each segment.

    Note:
        A demonstration of using this class can be seen
        in ``demo.py``
        https://github.com/RaivoKoot/Video-Dataset-Loading-Pytorch

    Note:
        This dataset broadly corresponds to the frame sampling technique
        introduced in ``Temporal Segment Networks`` at ECCV2016
        https://arxiv.org/abs/1608.00859.

    Args:
        root_path: The root path in which video folders lie.
                   this is ROOT_DATA from the description above.
        num_segments: The number of segments the video should
                      be divided into to sample frames from.
        frames_per_segment: The number of frames that should
                            be loaded per segment. For each segment's
                            frame-range, a random start index or the
                            center is chosen, from which frames_per_segment
                            consecutive frames are loaded.
        imagefile_template: The image filename template that video frame files
                            have inside of their video folders as described above.
        transform: Transform pipeline that receives a list of numpy images/frames.
        test_mode: If True, frames are taken from the center of each
                   segment, instead of a random location in each segment.

    """
    def __init__(self,
                 root_path: str,
                 num_segments: int = 3, # Possiamo adattare il modo per scegliere i frame dei video. Non si scelgono tutti i frami di un video in una rete.
                 frames_per_segment: int = 1, # Cambiando num_segments e frames_per_segment cambiamo il modo per scegliere i frame. NOTA: Ad ogni epoca scegliamo dei frame casuali per addestrare la rete. Dopo tante epoche probabilmente abbiamo preso tutti (o quasi) i frame del video.
                 # Prendo tre segmenti da un frame
                 imagefile_template: str='{:05d}.jpg', # è il pattern per i nomi del frame.
                 transform=None, # Sono tutte le augmentation e process che vogliamo applicare ai dati.
                 totensor=True, #
                 test_mode: bool = False): # Durante il training ha senso che prendiamo dei frame casuali dai segmenti -> test_mode = False. Ma durante la validation non ha senso cambiare il frame selezionato per valutare la loss -> test_mode = True.
        super(VideoFrameDataset, self).__init__()

        # Settiamo le variabili
        self.root_path = root_path
        self.num_segments = num_segments
        self.frames_per_segment = frames_per_segment
        self.imagefile_template = imagefile_template
        self.test_mode = test_mode

        if transform is None:
            self.transform = None
        else:
            additional_targets = {}
            for i in range(self.num_segments * self.frames_per_segment - 1):
                additional_targets["image%d" % i] = "image"
            self.transform = albumentations.Compose([transform],
                                                    additional_targets=additional_targets,
                                                    p=1)
        self.totensor = totensor
        self.totensor_transform = ImglistOrdictToTensor() # Applica toTensor ad un set di frame.

        self._parse_annotationfile() # Assegna le annotazioni ad ogni video
        self._sanity_check_samples() # Contorlla che le annotazioni ad ogni video sono corrette

    def _load_image(self, directory: str, idx: int) -> Image.Image:
        return np.asarray(Image.open(os.path.join(directory, self.imagefile_template.format(idx))).convert('RGB'))

    def _parse_annotationfile(self):
        self.video_list = []
        for class_name in os.listdir(self.root_path):
            for video_name in os.listdir(os.path.join(self.root_path, class_name)):
                frames_dir = os.path.join(self.root_path, class_name, video_name)
                if os.path.isdir(frames_dir):
                    frame_path = os.path.join(class_name, video_name)
                    end_frame = len(os.listdir(frames_dir))

                    # PARTE PIù IMPORTANTE. Questo codice ci permette di ottenere una corrispondenza fra video e annotation.
                    annotation_path = frames_dir\
                        .replace("\\", "/") \
                        .replace("FRAMES/", "GT/") \
                        .replace(".mp4", ".rtf")

                    with open(annotation_path, 'r') as file:
                        text = rtf_to_text(file.read())
                    if len(text):
                        label = 1
                        start_frame = int(text.split(",")[0])
                        if start_frame == 0:
                          start_frame = 1
                    else:
                        label = 0
                        start_frame = 1

                    self.video_list.append(VideoRecord( # Creo l'oggetto VideoRecord. Se nel training ho dei video con il fuoco ci sono frame che non hanno fuoco. Per evitare questo settiamo un frame iniziale dal quale inizia il fuoco e quindi dobbiamo campionare solo dopo questo frame -> è meglio scartare questi frame.
                        [frame_path, start_frame, end_frame, label],
                        self.root_path))

    def _sanity_check_samples(self):
        for record in self.video_list:
            if record.num_frames <= 0 or record.start_frame == record.end_frame:
                print(f"\nDataset Warning: video {record.path} seems to have zero RGB frames on disk!\n")

            elif record.num_frames < (self.num_segments * self.frames_per_segment):
                print(f"\nDataset Warning: video {record.path} has {record.num_frames} frames "
                      f"but the dataloader is set up to load "
                      f"(num_segments={self.num_segments})*(frames_per_segment={self.frames_per_segment})"
                      f"={self.num_segments * self.frames_per_segment} frames. Dataloader will throw an "
                      f"error when trying to load this video.\n")

    def _get_start_indices(self, record: VideoRecord) -> 'np.ndarray[int]':
        """
        For each segment, choose a start index from where frames
        are to be loaded from.

        Args:
            record: VideoRecord denoting a video sample.
        Returns:
            List of indices of where the frames of each
            segment are to be loaded from.
        """
        # choose start indices that are perfectly evenly spread across the video frames.
        if self.test_mode:
            distance_between_indices = (record.num_frames - self.frames_per_segment + 1) / float(self.num_segments)

            start_indices = np.array([int(distance_between_indices / 2.0 + distance_between_indices * x)
                                      for x in range(self.num_segments)])
        # randomly sample start indices that are approximately evenly spread across the video frames.
        else:
            max_valid_start_index = (record.num_frames - self.frames_per_segment + 1) // self.num_segments

            start_indices = np.multiply(list(range(self.num_segments)), max_valid_start_index) + \
                      np.random.randint(max_valid_start_index, size=self.num_segments)

        return start_indices

    def __getitem__(self, idx: int) -> Union[
        Tuple[List[Image.Image], Union[int, List[int]]],
        Tuple['torch.Tensor[num_frames, channels, height, width]', Union[int, List[int]]],
        Tuple[Any, Union[int, List[int]]],
        ]:
        """
        For video with id idx, loads self.NUM_SEGMENTS * self.FRAMES_PER_SEGMENT
        frames from evenly chosen locations across the video.

        Args:
            idx: Video sample index.
        Returns:
            A tuple of (video, label). Label is either a single
            integer or a list of integers in the case of multiple labels.
            Video is either 1) a list of PIL images if no transform is used
            2) a batch of shape (NUM_IMAGES x CHANNELS x HEIGHT x WIDTH) in the range [0,1]
            if the transform "ImglistToTensor" is used
            3) or anything else if a custom transform is used.
        """
        record: VideoRecord = self.video_list[idx]

        frame_start_indices: 'np.ndarray[int]' = self._get_start_indices(record)

        return self._get(record, frame_start_indices)

    def _get(self, record: VideoRecord, frame_start_indices: 'np.ndarray[int]') -> Union[
        Tuple[List[Image.Image], Union[int, List[int]]],
        Tuple['torch.Tensor[num_frames, channels, height, width]', Union[int, List[int]]],
        Tuple[Any, Union[int, List[int]]],
        ]:
        """
        Loads the frames of a video at the corresponding
        indices.

        Args:
            record: VideoRecord denoting a video sample.
            frame_start_indices: Indices from which to load consecutive frames from.
        Returns:
            A tuple of (video, label). Label is either a single
            integer or a list of integers in the case of multiple labels.
            Video is either 1) a list of PIL images if no transform is used
            2) a batch of shape (NUM_IMAGES x CHANNELS x HEIGHT x WIDTH) in the range [0,1]
            if the transform "ImglistToTensor" is used
            3) or anything else if a custom transform is used.
        """

        frame_start_indices = frame_start_indices + record.start_frame
        images = list()

        # from each start_index, load self.frames_per_segment
        # consecutive frames
        for start_index in frame_start_indices:
            frame_index = int(start_index)

            # load self.frames_per_segment consecutive frames
            for _ in range(self.frames_per_segment):
                image = self._load_image(record.path, frame_index)
                images.append(image)

                if frame_index < record.end_frame:
                    frame_index += 1

        if self.transform is not None:
            transform_input = {"image": images[0]}
            for i, image in enumerate(images[1:]):
                transform_input["image%d" % i] = image
            images = self.transform(**transform_input)

        if self.totensor:
            images = self.totensor_transform(images)
        return images, record.label

    def __len__(self):
        return len(self.video_list)


class ImglistOrdictToTensor(torch.nn.Module):
    """
    Converts a list or a dict of numpy images to a torch.FloatTensor
    of shape (NUM_IMAGES x CHANNELS x HEIGHT x WIDTH).
    Can be used as first transform for ``VideoFrameDataset``.
    """
    @staticmethod
    def forward(img_list_or_dict):
        """
        Converts each numpy image in a list or a dict to
        a torch Tensor and stacks them into a single tensor.

        Args:
            img_list_or_dict: list or dict of numpy images.
        Returns:
            tensor of size ``NUM_IMAGES x CHANNELS x HEIGHT x WIDTH``
        """
        if isinstance(img_list_or_dict, list):
            return torch.stack([transforms.functional.to_tensor(img)
                                for img in img_list_or_dict])
        else:
            return torch.stack([transforms.functional.to_tensor(img_list_or_dict[k])
                                for k in img_list_or_dict.keys()])


In [3]:
# Preprocessing and augmentation
preprocessing = albumentations.Sequential([
    albumentations.Resize(height=224, width=224, always_apply=True), # Facciamo la resize. Può succedere un pò di distorsione ma non è un problema.
    albumentations.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225],
                                max_pixel_value=255.,
                                always_apply=True),
])

augmentation = albumentations.OneOf([
    albumentations.HorizontalFlip(p=1.),
    ], p=.5)

# Dataset for training
train_dataset = VideoFrameDataset(root_path="FRAMES/TRAINING_SET/",
                            num_segments=3,
                            frames_per_segment=1,
                            transform=albumentations.Compose([
                                preprocessing,
                                augmentation],
                                p=1.,
                            )
                            )
# Dataset for validation
validation_dataset = VideoFrameDataset(root_path="FRAMES/VALIDATION_SET/",
                            num_segments=3,
                            frames_per_segment=1,
                            transform=albumentations.Compose([
                                preprocessing],
                                p=1.,)
                            )




print(train_dataset, "\nelements:", len(train_dataset))
X, y = train_dataset[0] # A tuple of (video, label). video: (NUM_IMAGES x CHANNELS x HEIGHT x WIDTH)

print('first element data', X.shape, X.min(), X.max(), X.mean(), X.std(), '\nlabel', y)

print(validation_dataset, "\nelements:", len(validation_dataset))
X, y = validation_dataset[0] # A tuple of (video, label). video: (NUM_IMAGES x CHANNELS x HEIGHT x WIDTH)

print('first element data', X.shape, X.min(), X.max(), X.mean(), X.std(), '\nlabel', y)



elements: 249
first element data torch.Size([3, 3, 224, 224]) tensor(-2.1008) tensor(2.6226) tensor(-0.9235) tensor(0.8918) 
label 0
elements: 63
first element data torch.Size([3, 3, 224, 224]) tensor(-1.9638) tensor(1.2385) tensor(-0.5446) tensor(0.7627) 
label 0


In [4]:

#dataloader_train = DataLoader(train_dataset, shuffle=True,batch_size=2, num_workers=4, pin_memory=True)
#dataloader_validation = DataLoader(validation_dataset, shuffle=True,batch_size=2, num_workers=4, pin_memory=True)
dataloader_train = DataLoader(train_dataset, shuffle=True,
                              batch_size=1, num_workers=1, pin_memory=True)
print(dataloader_train, "\nbatches:", len(dataloader_train))




for X, y in tqdm(dataloader_train):
  #counter +=1
  print('batch data', X.shape, X.min(), X.max(), X.mean(), X.std(), '\nlabel', y)

batches: 249


  6%|▌         | 14/249 [00:02<00:34,  6.85it/s]

batch data torch.Size([1, 3, 3, 224, 224]) tensor(-1.3815) tensor(2.0823) tensor(0.4796) tensor(0.8932) 
label tensor([1])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-2.0837) tensor(2.6400) tensor(0.9410) tensor(0.9645) 
label tensor([1])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-2.1179) tensor(2.6400) tensor(-0.0617) tensor(0.9945) 
label tensor([1])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-1.9809) tensor(2.4134) tensor(-0.1486) tensor(1.3176) 
label tensor([1])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-2.0323) tensor(2.6400) tensor(0.4251) tensor(1.2356) 
label tensor([0])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-2.1179) tensor(2.6400) tensor(-0.4173) tensor(1.1688) 
label tensor([1])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-2.1179) tensor(2.6400) tensor(0.4569) tensor(1.1545) 
label tensor([1])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-2.1179) tensor(2.6226) tensor(0.1153) tensor(0.6730) 
label tensor([1])
batch data to

 15%|█▍        | 37/249 [00:03<00:09, 22.34it/s]

batch data torch.Size([1, 3, 3, 224, 224]) tensor(-1.9638) tensor(2.6400) tensor(0.0277) tensor(1.4476) 
label tensor([0])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-1.9809) tensor(2.6400) tensor(0.0060) tensor(1.4518) 
label tensor([0])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-2.1179) tensor(2.6400) tensor(-0.2079) tensor(1.2464) 
label tensor([1])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-2.1179) tensor(2.6400) tensor(-1.2404) tensor(0.8556) 
label tensor([1])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-2.1179) tensor(2.6400) tensor(0.1219) tensor(1.0294) 
label tensor([1])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-2.0837) tensor(2.6400) tensor(-0.3703) tensor(1.1194) 
label tensor([0])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-2.0152) tensor(2.4483) tensor(-0.3039) tensor(1.2971) 
label tensor([1])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-2.1179) tensor(2.6400) tensor(-0.1148) tensor(1.1607) 
label tensor([0])
batch data 

...

tensor(2.0777) tensor(-0.4282) tensor(0.8184) 
label tensor([0])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-1.9638) tensor(2.0125) tensor(-0.0145) tensor(1.1892) 
label tensor([1])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-2.0837) tensor(2.6400) tensor(0.4020) tensor(0.8422) 
label tensor([0])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-1.8782) tensor(2.5703) tensor(0.1845) tensor(1.3243) 
label tensor([1])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-2.0152) tensor(1.3154) tensor(-0.5675) tensor(0.8859) 
label tensor([1])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-2.0152) tensor(1.2108) tensor(-0.5315) tensor(0.9341) 
label tensor([1])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-2.1179) tensor(2.6226) tensor(0.1282) tensor(0.6669) 
label tensor([1])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-2.1179) tensor(2.6400) tensor(-0.1355) tensor(0.9996) 
label tensor([1])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-1.9467) tensor(2.62

 98%|█████████▊| 244/249 [00:05<00:00, 93.08it/s] 

batch data torch.Size([1, 3, 3, 224, 224]) tensor(-2.1179) tensor(2.6400) tensor(0.1247) tensor(1.0301) 
label tensor([1])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-1.7240) tensor(2.0648) tensor(-0.0499) tensor(1.0321) 
label tensor([1])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-2.1179) tensor(2.6400) tensor(-0.3121) tensor(0.9257) 
label tensor([1])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-2.1008) tensor(2.6226) tensor(-0.8185) tensor(0.9369) 
label tensor([0])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-2.1179) tensor(2.6400) tensor(0.1152) tensor(0.9780) 
label tensor([1])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-2.1179) tensor(2.6400) tensor(0.1285) tensor(1.0338) 
label tensor([1])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-2.1179) tensor(2.6400) tensor(0.2623) tensor(1.4476) 
label tensor([1])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-1.9980) tensor(2.5703) tensor(-0.1772) tensor(1.2857) 
label tensor([0])
batch data t

100%|██████████| 249/249 [00:05<00:00, 48.07it/s]

tensor(-0.8399) tensor(0.8379) 
label tensor([1])
batch data torch.Size([1, 3, 3, 224, 224]) tensor(-2.0494) tensor(2.3761) tensor(-0.4625) tensor(1.2339) 
label tensor([1])


In [5]:
dataloader_validation = DataLoader(validation_dataset, shuffle=True,
                              batch_size=1, num_workers=1, pin_memory=True)
print(dataloader_train, "\nbatches:", len(dataloader_validation))

batches: 63


In [6]:
from torch.nn import Linear
def build_MobileNet(num_outputs=1):
  model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
  model.eval()

  #model.classifier[6] = Linear(4096, num_outputs)
  return model

model = build_MobileNet(1)
print(model)

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

Using cache found in /home/michele/.cache/torch/hub/pytorch_vision_v0.10.0
/home/michele/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/michele/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
#modello con sostituzione di ultimo layer con 2 output
from torch.nn import Linear,Sequential,Dropout 
def build_MobileNet(num_outputs=1): 
  model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True) 
  model.classifier = Sequential(Dropout(p=0.2, inplace=False),Linear(in_features=1280, out_features=num_outputs, bias=True)) 
  return model 
 
 
model = build_MobileNet(1) 
model.eval()

Using cache found in /home/michele/.cache/torch/hub/pytorch_vision_v0.10.0


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [8]:
from torch.utils.tensorboard import SummaryWriter
from tensorboard import notebook

def start_tensorboard(log_dir):
  writer = SummaryWriter(os.path.join("runs", log_dir))

  # run tensorboard in background
  ! killall tensorboard
  %load_ext tensorboard
  %tensorboard --logdir ./runs

  notebook.list() # View open TensorBoard instances

  return writer

In [9]:
from torchvision.utils import make_grid
from tqdm import tqdm

def one_epoch(model, lossFunction, output_activation, optimizer, train_loader, val_loader, writer, epoch_num):
  model.cuda()
  model.train()

  i_start = epoch_num * len(train_loader)
  for i, (X, y) in tqdm(enumerate(train_loader), desc="epoch {} - train".format(epoch_num)):
    (batch_size, frames, channels, width, height) = X.shape
    #print("\nprima: ",X.shape)
    #print("y: ",y)

    X = X.view(-1,channels, width, height)
    y = y.repeat(3)#.float()
    #print("dopo: ",X.shape)
    #print("y: ",y)

    if i == 0:
      writer.add_image('first_batch', make_grid(X))

    X = X.cuda()
    y = y.cuda().float()

    optimizer.zero_grad()

    o = model(X)
    o = output_activation(o).squeeze()
    l = lossFunction(o, y)


    l.backward()
    optimizer.step()

    acc = ((o.detach() > .5) == y.detach()).float().mean()

    print("- batch loss and accuracy : {:.7f}\t{:.4f}".format(l.detach().item(), acc))
    writer.add_scalar('train/loss', l.detach().item(), i_start+i)
    writer.add_scalar('train/acc', acc, i_start+i)

  model.eval()
  with torch.no_grad():
    val_loss = []
    val_corr_pred = []
    for X, y in tqdm(val_loader, desc="epoch {} - validation".format(epoch_num)):
      (batch_size, frames, channels, width, height) = X.shape
      X = X.view(-1,channels, width, height)
      y = y.repeat(3)#.float()

      X = X.cuda()
      y = y.cuda().float()  
      
      o = model(X)
      o = output_activation(o).squeeze()
      val_loss.append(lossFunction(o, y))
      val_corr_pred.append((o > .5) == y)

    val_loss = torch.stack(val_loss).mean().item()
    val_accuracy = torch.concatenate(val_corr_pred).float().mean().item()

    print("Validation loss and accuracy : {:.7f}\t{:.4f}".format(val_loss, val_accuracy))
    writer.add_scalar('val/loss', val_loss, i_start+i)
    writer.add_scalar('val/acc', val_accuracy, i_start+i)
  return val_loss, val_accuracy

In [10]:
from torch.nn import BCELoss, CrossEntropyLoss, Sigmoid, Softmax

# learning parameters
lossFunction, output_activation = BCELoss(), Sigmoid()
#batch_size = 512
lr = .001
momentum = .9
lambda_reg = 0

epochs = 200
early_stopping_patience = 40

# network parameters
for param in model.parameters():
  param.requires_grad = False
for param in model.classifier.parameters():
  param.requires_grad = True
optimizer = torch.optim.SGD(model.classifier.parameters(),
                          lr=lr,
                          weight_decay=lambda_reg,
                          momentum=momentum)

# create output directory and logger
experiment_name = "MobileNet_finetuning"
import os
dirs = os.listdir()

if experiment_name not in dirs:
  os.makedirs(experiment_name)
writer = start_tensorboard(experiment_name)

tensorboard: no process found


Known TensorBoard instances:
  - port 6006: logdir ./runs (started 0:00:00 ago; pid 1030)


In [11]:
from torch.utils.data import DataLoader
#######dataloader_params = {"batch_size": batch_size, "num_workers": 2, "pin_memory": True}
#######train_loader = DataLoader(train_dataset, shuffle=True, **dataloader_params)
######val_loader = DataLoader(val_dataset, shuffle=False, **dataloader_params)

# early stopping and best model saving
early_stopping_counter = early_stopping_patience
min_val_loss = 1e10

# training and validation
val_losses = torch.zeros(epochs)
val_accuracies = torch.zeros(epochs)
for e in range(epochs):
  print("EPOCH {}".format(e))
  ############val_loss, val_accuracy = one_epoch(model, lossFunction, output_activation, optimizer, train_loader, val_loader, writer, e)
  val_loss, val_accuracy = one_epoch(model, lossFunction, output_activation, optimizer, dataloader_train, dataloader_validation, writer, e)

  # store the validation metrics
  val_losses[e] = val_loss
  val_accuracies[e] = val_accuracy
  torch.save(val_losses, os.path.join(experiment_name,'val_losses.pth'))
  torch.save(val_accuracies, os.path.join(experiment_name,'val_accuracies.pth'))

  # save the best model and check the early stopping criteria
  if val_loss < min_val_loss: # save the best model
    min_val_loss = val_loss
    early_stopping_counter = early_stopping_patience # reset early stopping counter
    torch.save(model.state_dict(), os.path.join(experiment_name,'best_model.pth'))
    print("- saved best model: val_loss =", val_loss, "val_accuracy =", val_accuracy)

  if e>0: # early stopping counter update
    if val_losses[e] > val_losses[e-1]:
        early_stopping_counter -= 1 # update early stopping counter
    else:
        early_stopping_counter = early_stopping_patience # reset early stopping counter
  if early_stopping_counter == 0: # early stopping
      break

EPOCH 0


epoch 0 - train: 4it [00:03,  1.32it/s]

- batch loss and accuracy : 0.6977233	0.6667
- batch loss and accuracy : 0.6548787	0.6667
- batch loss and accuracy : 0.4368637	1.0000
- batch loss and accuracy : 0.3744050	1.0000
- batch loss and accuracy : 0.2170565	1.0000


epoch 0 - train: 15it [00:04,  7.65it/s]

- batch loss and accuracy : 2.3216534	0.0000
- batch loss and accuracy : 0.0994324	1.0000
- batch loss and accuracy : 0.1201925	1.0000
- batch loss and accuracy : 0.0880023	1.0000
- batch loss and accuracy : 2.8026600	0.0000
- batch loss and accuracy : 0.0763984	1.0000
- batch loss and accuracy : 0.0846164	1.0000
- batch loss and accuracy : 0.0925733	1.0000
- batch loss and accuracy : 0.1203619	1.0000
- batch loss and accuracy : 0.1042407	1.0000
- batch loss and accuracy : 2.5702586	0.0000
- batch loss and accuracy : 2.1608191	0.0000
- batch loss and accuracy : 1.7225213	0.0000
- batch loss and accuracy : 0.4074740	1.0000


...

epoch 41 - train: 140it [00:02, 50.12it/s]

- batch loss and accuracy : 0.0197158	1.0000
- batch loss and accuracy : 0.0286711	1.0000
- batch loss and accuracy : 0.0418393	1.0000
- batch loss and accuracy : 2.2584372	0.0000
- batch loss and accuracy : 1.3105779	0.0000
- batch loss and accuracy : 0.6050191	1.0000


epoch 41 - train: 144it [00:02, 50.61it/s]


- batch loss and accuracy : 0.4816725	0.6667


KeyboardInterrupt: 

In [ ]:
from torch.utils.data import DataLoader
#dataloader_params = {"batch_size": batch_size, "num_workers": 2, "pin_memory": True}
#val_loader = DataLoader(val_dataset, shuffle=False, **dataloader_params)

#model = build_AlexNet(1).cuda()
#model.load_state_dict(torch.load(os.path.join(experiment_name,'best_model.pth')))

Y, Y_hat = [], []
with torch.no_grad():
  model.eval()
  for X, y in dataloader_validation:
    Y.append(y)
    Y_hat.append((output_activation(model(X.cuda())) > .5).squeeze().cpu())

Y = torch.concatenate(Y)
Y_hat = torch.concatenate(Y_hat)
print("Validation accuracy:", (Y==Y_hat).float().mean().item())
from sklearn.metrics import confusion_matrix
confusion_matrix(Y, Y_hat)